# NASA EONET Events - Exploratory Analysis

This notebook uses DuckDB to analyze natural events data from NASA EONET.


In [2]:
import duckdb
import pandas as pd
from pathlib import Path

# Connect to DuckDB
con = duckdb.connect()

# Load the processed data
data_path = Path('../data/processed/events_2025.csv')

if not data_path.exists():
    print(f"Warning: {data_path} does not exist. Please run data_pipeline.py first.")
else:
    # Register the CSV as a table
    con.execute(f"CREATE OR REPLACE TABLE events AS SELECT * FROM read_csv_auto('{data_path.absolute()}')")
    print("Data loaded successfully!")


Data loaded successfully!


In [15]:
result1 = con.execute("select * from events limit 10").df()
result1


,event_id,event_title,event_description,event_link,category_ids,category_titles,occurrence_date,occurrence_type,longitude,latitude,year,month,day
0,EONET_15859,Typhoon Fung-wong,None,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,severeStorms,Severe Storms,2025-11-05 21:00:00-03:00,Point,140.8,9.6,2025,11,6
1,EONET_15859,Typhoon Fung-wong,None,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,severeStorms,Severe Storms,2025-11-06 03:00:00-03:00,Point,140.6,9.7,2025,11,6
2,EONET_15859,Typhoon Fung-wong,None,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,severeStorms,Severe Storms,2025-11-06 09:00:00-03:00,Point,139.6,10.2,2025,11,6
3,EONET_15859,Typhoon Fung-wong,None,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,severeStorms,Severe Storms,2025-11-06 15:00:00-03:00,Point,138.6,10.7,2025,11,6
4,EONET_15859,Typhoon Fung-wong,None,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,severeStorms,Severe Storms,2025-11-06 21:00:00-03:00,Point,137.9,11.4,2025,11,7
5,EONET_15859,Typhoon Fung-wong,None,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,severeStorms,Severe Storms,2025-11-07 03:00:00-03:00,Point,136.6,11.6,2025,11,7
6,EONET_15859,Typhoon Fung-wong,None,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,severeStorms,Severe Storms,2025-11-07 09:00:00-03:00,Point,135.4,11.9,2025,11,7
7,EONET_15859,Typhoon Fung-wong,None,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,severeStorms,Severe Storms,2025-11-07 15:00:00-03:00,Point,133.8,12.2,2025,11,7
8,EONET_15859,Typhoon Fung-wong,None,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,severeStorms,Severe Storms,2025-11-07 21:00:00-03:00,Point,132.1,12.5,2025,11,8
9,EONET_15859,Typhoon Fung-wong,None,https://eonet.gsfc.nasa.gov/api/v3/events/EONE...,severeStorms,Severe Storms,2025-11-08 03:00:00-03:00,Point,130.6,13.0,2025,11,8


## Question 1: What was the most frequent event category in 2024?


In [3]:
query1 = """
SELECT 
    category_title,
    COUNT(*) as event_count
FROM (
    SELECT 
        UNNEST(STRING_SPLIT(category_titles, ',')) as category_title
    FROM events
    WHERE year = 2024
)
GROUP BY category_title
ORDER BY event_count DESC
LIMIT 1
"""

result1 = con.execute(query1).df()
print("Most frequent event category in 2024:")
print(result1)


Most frequent event category in 2024:
  category_title  event_count
0      Wildfires         4114


## Question 2: What was the distribution of events by category in 2024?


In [4]:
query2 = """
SELECT 
    category_title,
    COUNT(*) as event_count,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentage
FROM (
    SELECT 
        UNNEST(STRING_SPLIT(category_titles, ',')) as category_title
    FROM events
    WHERE year = 2024
)
GROUP BY category_title
ORDER BY event_count DESC
"""

result2 = con.execute(query2).df()
print("Distribution of events by category in 2024:")
print(result2)


Distribution of events by category in 2024:
     category_title  event_count  percentage
0         Wildfires         4114       97.56
1  Sea and Lake Ice           96        2.28
2         Volcanoes            7        0.17


## Question 3: How many occurrences were there per month in 2024?


In [5]:
query3 = """
SELECT 
    month,
    CASE month
        WHEN 1 THEN 'January'
        WHEN 2 THEN 'February'
        WHEN 3 THEN 'March'
        WHEN 4 THEN 'April'
        WHEN 5 THEN 'May'
        WHEN 6 THEN 'June'
        WHEN 7 THEN 'July'
        WHEN 8 THEN 'August'
        WHEN 9 THEN 'September'
        WHEN 10 THEN 'October'
        WHEN 11 THEN 'November'
        WHEN 12 THEN 'December'
    END as month_name,
    COUNT(*) as occurrence_count
FROM events
WHERE year = 2024 AND occurrence_date IS NOT NULL
GROUP BY month
ORDER BY month
"""

result3 = con.execute(query3).df()
print("Occurrences per month in 2024:")
print(result3)


Occurrences per month in 2024:
    month month_name  occurrence_count
0       1    January                12
1       2   February                22
2       3      March                46
3       4      April                19
4       5        May                87
5       6       June               558
6       7       July               931
7       8     August              1115
8       9  September               679
9      10    October               332
10     11   November               169
11     12   December               247


## Question 4: Which week had the most occurrences in 2024?


In [6]:
query4 = """
SELECT 
    YEAR(occurrence_date) as year,
    WEEK(occurrence_date) as week_number,
    MIN(occurrence_date) as week_start,
    COUNT(*) as occurrence_count
FROM events
WHERE year = 2024 AND occurrence_date IS NOT NULL
GROUP BY YEAR(occurrence_date), WEEK(occurrence_date)
ORDER BY occurrence_count DESC
LIMIT 1
"""

result4 = con.execute(query4).df()
print("Week with the most occurrences in 2024:")
print(result4)


Week with the most occurrences in 2024:
   year  week_number                week_start  occurrence_count
0  2024           32 2024-08-05 09:24:00-04:00               361


## Question 5: For each category, which month had the most events?


In [8]:
query5 = """
WITH category_month_counts AS (
    SELECT 
        UNNEST(STRING_SPLIT(category_titles, ',')) as category_title,
        month,
        CASE month
            WHEN 1 THEN 'January'
            WHEN 2 THEN 'February'
            WHEN 3 THEN 'March'
            WHEN 4 THEN 'April'
            WHEN 5 THEN 'May'
            WHEN 6 THEN 'June'
            WHEN 7 THEN 'July'
            WHEN 8 THEN 'August'
            WHEN 9 THEN 'September'
            WHEN 10 THEN 'October'
            WHEN 11 THEN 'November'
            WHEN 12 THEN 'December'
        END as month_name,
        COUNT(*) as event_count
    FROM events
    WHERE year = 2024
    GROUP BY category_title, month
),
ranked_months AS (
    SELECT 
        category_title,
        month,
        month_name,
        event_count,
        ROW_NUMBER() OVER (PARTITION BY category_title ORDER BY event_count DESC) as rn
    FROM category_month_counts
)
SELECT 
    category_title,
    month_name,
    event_count
FROM ranked_months
WHERE rn = 1
ORDER BY category_title
"""

result5 = con.execute(query5).df()
print("Month with most events for each category in 2024:")
print(result5)


BinderException: Binder Error: UNNEST not supported here

LINE 4:         UNNEST(STRING_SPLIT(category_titles, ',')) as category_title...
                ^

## Question 6: In which countries or continents were the largest number of events concentrated in 2024?

Note: This requires reverse geocoding. For now, we'll analyze by geographic regions based on coordinates.


In [ ]:
# For a more accurate analysis, we would need reverse geocoding.
# For now, we'll analyze by geographic regions based on coordinates
query6 = """
SELECT 
    CASE 
        WHEN latitude BETWEEN -90 AND -23.5 THEN 'Southern Hemisphere (Temperate)'
        WHEN latitude BETWEEN -23.5 AND 0 THEN 'Southern Hemisphere (Tropical)'
        WHEN latitude BETWEEN 0 AND 23.5 THEN 'Northern Hemisphere (Tropical)'
        WHEN latitude BETWEEN 23.5 AND 90 THEN 'Northern Hemisphere (Temperate)'
        ELSE 'Unknown'
    END as region,
    CASE
        WHEN longitude BETWEEN -180 AND -30 THEN 'Americas'
        WHEN longitude BETWEEN -30 AND 60 THEN 'Europe/Africa'
        WHEN longitude BETWEEN 60 AND 180 THEN 'Asia/Pacific'
        ELSE 'Unknown'
    END as longitude_region,
    COUNT(*) as event_count
FROM events
WHERE year = 2024 AND latitude IS NOT NULL AND longitude IS NOT NULL
GROUP BY region, longitude_region
ORDER BY event_count DESC
"""

result6 = con.execute(query6).df()
print("Event concentration by geographic regions in 2024:")
print(result6)

# Close the connection
con.close()
